In [108]:
import asyncio
import io
import glob
import os
import sys
import time
import uuid
import requests
from urllib.parse import urlparse
from io import BytesIO
from PIL import Image, ImageDraw
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.vision.face.models import TrainingStatusType, Person, SnapshotObjectType, OperationStatusType


In [37]:
!pip install --upgrade azure-cognitiveservices-vision-face

Requirement already up-to-date: azure-cognitiveservices-vision-face in c:\users\tamittal\appdata\local\continuum\anaconda3\lib\site-packages (0.4.0)


In [38]:
os.getcwd()

'C:\\Users\\tamittal\\source\\Untitled Folder'

In [41]:
os.environ['FACE_ENDPOINT']

'https://facedetectioncapstone.cognitiveservices.azure.com/'

In [109]:
# Set the FACE_SUBSCRIPTION_KEY environment variable with your key as the value.
# This key will serve all examples in this document.
KEY = os.environ['FACE_SUBSCRIPTION_KEY']
#3e57e8993cfb4888863a13cf0d48cf85


# Set the FACE_ENDPOINT environment variable with the endpoint from your Face service in Azure.
# This endpoint will be used in all examples in this quickstart.
ENDPOINT = os.environ['FACE_ENDPOINT']
#https://facedetectioncapstone.cognitiveservices.azure.com/

In [110]:
# Create an authenticated FaceClient.
face_client = FaceClient(ENDPOINT, CognitiveServicesCredentials(KEY))

In [111]:
# Used in the Person Group Operations,  Snapshot Operations, and Delete Person Group examples.
# You can call list_person_groups to print a list of preexisting PersonGroups.
# SOURCE_PERSON_GROUP_ID should be all lowercase and alphanumeric. For example, 'mygroupname' (dashes are OK).
PERSON_GROUP_ID = 'family'

In [113]:
#Create Person Group
print('Person group:', PERSON_GROUP_ID)
face_client.person_group.create(person_group_id=PERSON_GROUP_ID, name=PERSON_GROUP_ID)

# Create Person in taht group
bean = face_client.person_group_person.create(PERSON_GROUP_ID, "bean")
shah = face_client.person_group_person.create(PERSON_GROUP_ID, "shah")
print(bean)

Person group: family
{'additional_properties': {}, 'name': None, 'user_data': None, 'person_id': '2a785130-ffaf-453e-bafe-856c4f0e977d', 'persisted_face_ids': None}


In [148]:
bean_images = [file for file in glob.glob('*.jfif') if file.startswith("download")]

In [87]:
bean_images

['download (1).jfif', 'download (2).jfif', 'download.jfif']

In [149]:
for image in bean_images:
    b = open(image, 'r+b')
    face_client.person_group_person.add_face_from_stream(PERSON_GROUP_ID, bean.person_id, b)
    print(face_client)

In [114]:
shah_images = [file for file in glob.glob('*.jfif') if file.startswith("shah")]

In [117]:
shah_images

['shah1.jfif', 'shah2.jfif', 'shah3.jfif']

In [118]:
for image in shah_images:
    s = open(image, 'r+b')
    face_client.person_group_person.add_face_from_stream(PERSON_GROUP_ID, shah.person_id, s)
    print(face_client)

In [119]:
'''
Train PersonGroup
'''
print()
print('Training the person group...')
# Train the person group
face_client.person_group.train(PERSON_GROUP_ID)

while (True):
    training_status = face_client.person_group.get_training_status(PERSON_GROUP_ID)
    print("Training status: {}.".format(training_status.status))
    print()
    if (training_status.status is TrainingStatusType.succeeded):
        break
    elif (training_status.status is TrainingStatusType.failed):
        sys.exit('Training the person group has failed.')
    time.sleep(5)


Training the person group...
Training status: running.

Training status: succeeded.



In [168]:
'''
Identify a face against a defined PersonGroup
'''
# Group image for testing against
group_photo = 'srk.jfif'
IMAGES_FOLDER=os.getcwd()
#IMAGES_FOLDER = os.path.join(os.path.dirname(os.path.realpath(__file__)))
# Get test image
test_image_array = glob.glob(os.path.join(IMAGES_FOLDER, group_photo))
image = open(test_image_array[0], 'r+b')
#print(image)
# Detect faces
face_ids = []
faces = face_client.face.detect_with_stream(image)
#print(faces)
for face in faces:
    print(face.face_id)
    face_ids.append(face.face_id)

ee0353b7-5371-4127-aaaa-f9264744aef6
37ce841e-c478-43f5-a787-6fd425056b87
4b7523e5-2a04-4e72-a291-71d370d3a5ef


In [169]:

count=0
# Identify faces
#print(face_ids, PERSON_GROUP_ID)
results = face_client.face.identify(face_ids, PERSON_GROUP_ID)
print(results[1])
print('Identifying faces in {}'.format(os.path.basename(image.name)))
if not results:
    print('No person identified in the person group for faces from {}.'.format(os.path.basename(image.name)))
for person in results: 
    if (person.candidates== []) :
        print("Can not identify this person for face ID {}".format(person.face_id))
        count=count+10
        continue;
    #print(person)
    #print(person.face_id)
    #print(os.path.basename(image.name))
    #print(person)
    print('Person for face ID {} is identified in {} with a confidence of {}.'.format(person.face_id, os.path.basename(image.name), person.candidates[0].confidence)) # Get topmost confidence score
print(count)

{'additional_properties': {}, 'face_id': '37ce841e-c478-43f5-a787-6fd425056b87', 'candidates': []}
Identifying faces in srk.jfif
Can not identify this person for face ID ee0353b7-5371-4127-aaaa-f9264744aef6
Can not identify this person for face ID 37ce841e-c478-43f5-a787-6fd425056b87
Person for face ID 4b7523e5-2a04-4e72-a291-71d370d3a5ef is identified in srk.jfif with a confidence of 0.7569.
20
